In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rev-padded/Reverse_padded/x_train1332.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1839.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1407.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1762.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1633.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1052.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1585.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1487.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1543.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1325.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1453.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1262.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1454.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1913.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1960.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1939.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1832.npy
/kaggle/input/rev-padded/Reverse_padded/x_train1

In [4]:
import numpy as np
from sklearn.utils import shuffle
import json

def get_data():
    lst_x = []
    lst_y = list(np.load("../input/mlsp-padded/reshaped/y_train.npy"))
    with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
        dict1 = json.loads(f.read()) 
    for m in range(1000):
        lst_x.append(np.load("../input/mlsp-padded/reshaped/x_train" + str(m) + ".npy"))
    return lst_x, lst_y, dict1


def train_val_split(lst1, lst_y): # input shapes are: (1000, 128, 2584) and (1000, 10)
    count = 1
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    for m in range(len(lst1)):
        if m <= (count*100) - 21:
            x_train.append(lst1[m])
            y_train.append(lst_y[m])
        else:
            x_test.append(lst1[m])
            y_test.append(lst_y[m])
        if m > (count*100) - 2:
            count = count + 1
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test =  np.array(y_test)
    del(lst1)
    x_train, y_train = shuffle(x_train, y_train)
    x_test, y_test = shuffle(x_test, y_test)
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    return x_train, x_test, y_train, y_test

In [3]:
x_train, y_train, dict1 = get_data()

In [ ]:
print(len(x_train))

In [5]:
x_train, x_test, y_train, y_test = train_val_split(x_train, y_train)

In [6]:
print(x_test.shape)
print(y_train.shape)
print(dict1)

(200, 128, 2584, 1)
(800, 10)
{'Bark': 0, 'Meow': 1, 'Siren': 2, 'Shatter': 3, 'Knock': 4, 'Crying_and_sobbing': 5, 'Microwave_oven': 6, 'Vehicle_horn_and_car_horn_and_honking': 7, 'Doorbell': 8, 'Walk_and_footsteps': 9}


In [7]:
(temp, f, t, num) = x_train.shape
print(f)
print(t)
print(num)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

128
2584
1
(800, 128, 2584, 1)
(200, 128, 2584, 1)
(800, 10)
(200, 10)


In [8]:
N_classes = 10
def make_model(f, t, num):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(f, t, num)))
    model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='leaky_relu'))
    #model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='leaky_relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(N_classes, activation='softmax'))
    return model
model = make_model(f, t, num)

2022-10-11 10:57:20.621874: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 10:57:20.717733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 10:57:20.718559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 10:57:20.719903: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 1292, 32)      320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 1292, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 646, 32)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 646, 32)       128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 646, 64)       18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 646, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 323, 64)       0

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

# Train model for 40 epochs, capture the history
history = model.fit(x_train, y_train, verbose = 1, epochs=40, validation_data=(x_test, y_test))

2022-10-11 10:58:14.653779: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1058406400 exceeds 10% of free system memory.
2022-10-11 10:58:15.846730: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1058406400 exceeds 10% of free system memory.
2022-10-11 10:58:16.730942: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2022-10-11 10:58:18.790589: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


25/25 [==============================] - 12s 142ms/step - loss: 2.3357 - precision: 0.4387 - recall: 0.1163 - val_loss: 2.0829 - val_precision: 0.6757 - val_recall: 0.1250
Epoch 2/40
25/25 [==============================] - 2s 90ms/step - loss: 1.6565 - precision: 0.6973 - recall: 0.2275 - val_loss: 1.8805 - val_precision: 0.6087 - val_recall: 0.0700
Epoch 3/40
25/25 [==============================] - 2s 89ms/step - loss: 1.3084 - precision: 0.7822 - recall: 0.3413 - val_loss: 1.9265 - val_precision: 1.0000 - val_recall: 0.0300
Epoch 4/40
25/25 [==============================] - 2s 92ms/step - loss: 1.0049 - precision: 0.8608 - recall: 0.5100 - val_loss: 2.0715 - val_precision: 1.0000 - val_recall: 0.0100
Epoch 5/40
25/25 [==============================] - 2s 89ms/step - loss: 0.6215 - precision: 0.9328 - recall: 0.6938 - val_loss: 1.9642 - val_precision: 1.0000 - val_recall: 0.0100
Epoch 6/40
25/25 [==============================] - 2s 88ms/step - loss: 0.4183 - precision: 0.9462 - re

In [11]:
model.save('model_1.h5')

In [12]:
yhat_probs = model.predict(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
y_target = []
for m in y_test:
    r = np.where(m == 1)
    y_target.append(int(r[0]))

7/7 [==============================] - 0s 29ms/step
[8 7 7 9 8 5 1 1 8 7 2 1 7 8 6 5 4 2 7 9 7 1 2 2 1 4 0 0 8 6 1 7 6 4 3 0 4
 9 0 3 7 0 5 5 3 3 0 1 3 1 7 4 9 9 4 6 1 2 9 0 1 8 2 8 8 9 6 1 2 8 5 4 4 4
 6 6 9 2 4 5 4 4 2 8 7 4 5 1 5 5 0 5 2 3 7 3 9 0 3 7 8 5 6 2 9 3 1 5 8 3 7
 1 7 7 6 1 7 0 5 4 6 5 0 8 6 4 0 2 9 4 5 8 4 5 0 5 6 0 2 9 0 6 7 8 0 1 8 9
 6 4 3 0 7 4 4 5 5 6 3 6 4 2 4 1 0 8 9 0 4 7 4 2 3 3 4 9 6 9 7 3 1 2 5 6 6
 2 7 9 5 1 7 0 1 6 8 3 7 1 7 8]


In [13]:
print(len(y_hat))
print(len(y_target))

200
200


In [40]:
from sklearn.metrics import classification_report
report = classification_report(y_target, y_hat, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [41]:
lst = []
for m in range(10):
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)
lst.append("accuracy")
lst.append("macro avg")
lst.append("weighted avg")
report_df["Labels"] = lst

In [42]:
report_df

,precision,recall,f1-score,support,Labels
0,0.850000,0.850,0.850000,20.000,Bark
1,0.761905,0.800,0.780488,20.000,Meow
2,1.000000,0.850,0.918919,20.000,Siren
3,0.812500,0.650,0.722222,20.000,Shatter
4,0.640000,0.800,0.711111,20.000,Knock
5,0.761905,0.800,0.780488,20.000,Crying_and_sobbing
6,0.800000,0.800,0.800000,20.000,Microwave_oven
7,0.791667,0.950,0.863636,20.000,Vehicle_horn_and_car_horn_and_honking
8,1.000000,0.950,0.974359,20.000,Doorbell
9,0.705882,0.600,0.648649,20.000,Walk_and_footsteps


In [43]:
report_df.to_csv("model_1_result.csv")